# Perf Tools Analisys

Neste notebook iremos verificar as analises referentes ao perf tools da implementação do loop fusion, verificando quanto de acesso ao cache misses e cache references é utilizado.

In [150]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import datetime
import time
from IPython.display import display
from collections import Counter
import os
import re
import math
import random
pd.set_option('display.max_rows', 300)
pd.options.display.float_format = '{:,.2f}'.format

In [151]:
#loads the csv as a dataframe and standardizes the algorithm names 
def load_df(filename):
    df = pd.read_csv(filename, index_col=False)
    return select_columns_and_rename_values(df)

In [152]:
#filters by substring (there are multiple OzaBag algorithms)
def filter_by_substring_algorithm(df, string):
    aux = df[df['algorithm'].str.contains(string, regex=False)]
    ret = aux
    if string == 'OB':
        ret = aux[~aux.algorithm.str.contains("Adwin|ASHT")]
    elif string == 'OzaBag':
        ret = aux[(aux.algorithm.str.contains(string)) & (~aux.algorithm.str.contains("Adwin|ASHT"))]
    return ret

In [174]:
#standardize algorithm names
def select_columns_and_rename_values(df):
    df.algorithm = df.algorithm.str.replace("Executor", "")
    df['algorithm'] = df["algorithm"].str.replace("OzaBag", "OB")
    df['algorithm'] = df["algorithm"].str.replace("AdaptiveRandomForest", "ARF")
    df['algorithm'] = df["algorithm"].str.replace("SequentialChunk", "SeqMB")
    df['algorithm'] = df["algorithm"].str.replace("OB$", "OBSequential")
    df['algorithm'] = df['algorithm'].str.replace("LeveragingBag", "LBagSequential")
    df['algorithm'] = df['algorithm'].str.replace("Adwin$", "AdwinSequential")
    df['algorithm'] = df['algorithm'].str.replace("CHUNK", "MB")
    df['algorithm'] = df['algorithm'].str.replace("MAXChunk", "MB")
    df['algorithm'] = df['algorithm'].str.replace("StreamingRandomPatches", "SRP")
    df['algorithm'] = df['algorithm'].str.replace("SRP$", "SRPSequential")
    df['algorithm'] = df['algorithm'].str.replace("OBASHT$", "OBASHTSequential")
    df.batch_size.unique()
    return df

In [175]:
#Folder inside results directory that contains all the MOA dump files for these experiments
%cd /Users/reginaldoluisdeluna/Documents/Ufscar/comparison-xue3m-minibatching/
folderMOADumps = "/Users/reginaldoluisdeluna/Documents/Ufscar/comparison-xue3m-minibatching/results/sem-coletor/mini-batching/perf/first"
wantedCSVfilename = "loop-fusion-perf"

/Users/reginaldoluisdeluna/Documents/Ufscar/comparison-xue3m-minibatching


In [176]:
import re
MILLION = 1000000
def parse_perf_file(fname, methodology='MB'):
    spname = fname.split('/')[-1].split('-')

    lineDict = {
        'dataset': spname[1],
        'algorithm': spname[2],
        'ensemble': spname[3],
        'methodology': methodology,
        'cores': 4,
        'batch_size': spname[5],
        'rate': spname[6],
        'cache-misses': None,
        'cache-references': None
    }

    with open (fname) as file:
        cache_misses = "0"
        cache_references = "0"
        for line in file:
            line = re.findall(r'\S+', line)
            try:
                if line[1]:
                    if line[1] == "cache-misses:u":
                        cache_misses = line[0]
                        
                    if line[1] == "cache-references:u":
                        cache_references = line[0]
    
                    response.append(lineDict)
            except IndexError:
                pass

    lineDict['cache-misses'] = float(cache_misses.replace(',', '')) / MILLION
    lineDict['cache-references'] = float(cache_references.replace(',', '')) / MILLION
            
    return lineDict

In [177]:
resultsFolder = f"{folderMOADumps}"
csvFile = f"parsed_csvs/{wantedCSVfilename}"
directory = os.fsencode(resultsFolder)
header_printed = False

response_mb = []
with open(f"{csvFile}", "w+") as output:
    for file in os.listdir(directory):
        filename = os.fsdecode(file)
        if filename.startswith("perf-"):
            response_mb.append(
                parse_perf_file(f'{os.fsdecode(directory)}/{filename}', 'MB')
            )

df_mini_batching = pd.DataFrame.from_dict(response_mb)
display(df_mini_batching.sort_values(by=['dataset', 'algorithm']).drop_duplicates())

,dataset,algorithm,ensemble,methodology,cores,batch_size,rate,cache-misses,cache-references
17,GMSC,AdaptiveRandomForestExecutorMAXChunk,25,MB,4,50,1,"1,754.56","66,333.15"
9,GMSC,LBagExecutorMAXChunk,25,MB,4,50,1,"1,445.82","55,310.77"
12,GMSC,OzaBagASHTExecutorMAXChunk,25,MB,4,50,1,568.48,"17,763.95"
1,GMSC,OzaBagAdwinExecutorMAXChunk,25,MB,4,50,1,554.64,"22,920.63"
21,GMSC,OzaBagExecutorMAXChunk,25,MB,4,50,1,655.21,"18,911.13"
18,GMSC,StreamingRandomPatchesExecutorMAXChunk,25,MB,4,50,1,"2,072.19","75,241.20"
22,airlines,AdaptiveRandomForestExecutorMAXChunk,25,MB,4,50,1,0.00,0.00
7,airlines,LBagExecutorMAXChunk,25,MB,4,50,1,"2,334.17","93,299.55"
23,airlines,OzaBagASHTExecutorMAXChunk,25,MB,4,50,1,"1,850.16","85,953.64"
3,airlines,OzaBagAdwinExecutorMAXChunk,25,MB,4,50,1,"2,199.84","92,043.68"


In [157]:
#Folder inside results directory that contains all the MOA dump files for these experiments
%cd /Users/reginaldoluisdeluna/Documents/Ufscar/comparison-xue3m-minibatching/
folderMOADumps = "/Users/reginaldoluisdeluna/Documents/Ufscar/comparison-xue3m-minibatching/results/loop-fusion/loop-fusion-perf/second"
wantedCSVfilename = "loop-fusion-perf"

/Users/reginaldoluisdeluna/Documents/Ufscar/comparison-xue3m-minibatching


In [158]:
resultsFolder = f"{folderMOADumps}"
csvFile = f"parsed_csvs/{wantedCSVfilename}"
directory = os.fsencode(resultsFolder)
header_printed = False

response = []
with open(f"{csvFile}", "w+") as output:
    for file in os.listdir(directory):
        filename = os.fsdecode(file)
        if filename.startswith("perf-"):
            response.append(
                parse_perf_file(f'{os.fsdecode(directory)}/{filename}', 'MB-LF')
            )

df = pd.DataFrame.from_dict(response)
display(df.sort_values(by=['dataset', 'algorithm']).drop_duplicates())

,dataset,algorithm,ensemble,methodology,cores,batch_size,rate,cache-misses,cache-references
464,GMSC,AdaptiveRandomForestExecutorMAXChunk,25,MB-LF,4,50,1,"1,854.57","72,717.01"
145,GMSC,LBagExecutorMAXChunk,25,MB-LF,4,50,1,"1,378.60","57,385.89"
29,GMSC,OzaBagASHTExecutorMAXChunk,25,MB-LF,4,50,1,360.43,"12,057.65"
290,GMSC,OzaBagAdwinExecutorMAXChunk,25,MB-LF,4,50,1,547.88,"23,702.32"
523,GMSC,OzaBagExecutorMAXChunk,25,MB-LF,4,50,1,400.55,"12,859.90"
435,GMSC,StreamingRandomPatchesExecutorMAXChunk,25,MB-LF,4,50,1,"3,656.74","141,587.31"
522,airlines,AdaptiveRandomForestExecutorMAXChunk,25,MB-LF,4,50,1,0.00,0.00
58,airlines,LBagExecutorMAXChunk,25,MB-LF,4,50,1,"61,329.64","2,574,339.12"
493,airlines,OzaBagASHTExecutorMAXChunk,25,MB-LF,4,50,1,"1,752.40","150,673.11"
261,airlines,OzaBagAdwinExecutorMAXChunk,25,MB-LF,4,50,1,"17,816.01","820,965.91"


In [178]:
#Folder inside results directory that contains all the MOA dump files for these experiments
%cd /Users/reginaldoluisdeluna/Documents/Ufscar/comparison-xue3m-minibatching/
folderMOADumps = "/Users/reginaldoluisdeluna/Documents/Ufscar/comparison-xue3m-minibatching/results/loop-fusion/sequential-perf/first"
wantedCSVfilename = "sequential-perf"

/Users/reginaldoluisdeluna/Documents/Ufscar/comparison-xue3m-minibatching


In [179]:
resultsFolder = f"{folderMOADumps}"
csvFile = f"parsed_csvs/{wantedCSVfilename}"
directory = os.fsencode(resultsFolder)
header_printed = False

response = []
with open(f"{csvFile}", "w+") as output:
    for file in os.listdir(directory):
        filename = os.fsdecode(file)
        if filename.startswith("perf-"):
            response.append(
                parse_perf_file(f'{os.fsdecode(directory)}/{filename}', 'Sequential')
            )

df = pd.DataFrame.from_dict(response)
display(df.sort_values(by=['dataset', 'algorithm']).drop_duplicates())

,dataset,algorithm,ensemble,methodology,cores,batch_size,rate,cache-misses,cache-references
580,GMSC,AdaptiveRandomForestSequential,25,Sequential,4,1,1,"1,061.92","35,398.63"
174,GMSC,LeveragingBag,25,Sequential,4,1,1,"1,045.04","32,457.90"
203,GMSC,OzaBag,25,Sequential,4,1,1,716.21,"16,950.50"
377,GMSC,OzaBagASHT,25,Sequential,4,1,1,701.86,"17,968.04"
522,GMSC,OzaBagAdwin,25,Sequential,4,1,1,828.72,"22,116.55"
406,GMSC,StreamingRandomPatches,25,Sequential,4,1,1,"1,109.62","37,295.12"
290,airlines,AdaptiveRandomForestSequential,25,Sequential,4,1,1,"1,152.11","48,376.35"
493,airlines,LeveragingBag,25,Sequential,4,1,1,"1,306.81","46,331.21"
0,airlines,OzaBag,25,Sequential,4,1,1,"1,224.42","45,372.62"
58,airlines,OzaBagASHT,25,Sequential,4,1,1,"1,158.25","45,298.75"
